# Lesson 1: Pattern Discovery Basic Concepts

## Frequent Patterns and Association Rules
* itemset: A set of one or more items
* k-itemset: X = {x1, ..., xk}
* (absolute) support (count) of X: Frequency or the number of occurrences of an itemset X
* (relative) support, s: The fraction of transactions that contains X (i.e. the probability)
* An itemset X is **frequent** if the support of X is no less than a **minsup** threshold (denoted as $\sigma$)
* Association rules: X --> Y (s, c)
    * Support, s: the p that a transation contains X $\cup$ Y
    * Confidence, c: the conditional p that a transation containing X also contains Y
        * c = sup(X $\cup$ Y) / sup(X)
* Association Rule Mining: Find all the rules, X --> Y with min sup and con

## Challenge: There are too many frequent patterns!
* Given TDB: T_1:{a_1,..., a_50}, T_2:{a_1,..., a_100}
    * Solution 1: Closed patterns: A pattern (itemset) X is **closed** if X is **frequent** and there exists no **super-pattern** Y contains X, with the same **support** as X.
        * Super patterns by definition will have lower or equal support than their patterns.
            * ex: (a, b) has p = .5, (a, b, c) has p = .5. Not closed.
            * ex: (a, b, c) has p = .5, (a, b, c, d) has p = .25. Closed.
    * Solution 2: Max-patterns: A pattern X is **max-pattern** if X is frequent and there exits no frequent super-pattern Y contains X.
        * This is very lossy because you may lose frequent patterns
            * ex: (a, b) has p = .5, (a, b, c) has p = .4, both are frequent. Only (a, b, c) is the max-pattern.

# Lesson 2: Efficient Pattern Mining Methods

## The Downward Closure Property of Frequent Patterns

The **downward closure (aka "Apriori")** property of frequent patterns status any subset of a frequent itemset must be frequent. Likewise if any itemset is infrequent the **Apriori pruning principle** states that any itemset that's infrequent's superset shouldn't be included.

** Apriori Algorithm **
1. Scen DB once to get frequent 1-itemset
2. Generate length-(k+1) candidate itemsets from length-k frequent it itemsets
3. Test the candidates against DB to find frequent (k+1)-itemsets
4. Set k := k + 1
5. Repreat 2-4 until no frequent or candidate set can be generated
6. Return all frequent itemsets.

In [ ]:
def apriori(data, f):
    # data needs to be a pd.DataFrame in the following shape:
    # tid set
    # 0 {a, b, c}
    
    total_len = len(data)
    pd.value_counts()

In order to speed up the Apriori Algorithm:
* **Partitioning**
    * Theorem: Any itemset that is potentially frequent in TDB must be frequent in at least one of the partitions of TDB
    * Method
        * Scan 1: Partition db and find local frequent patterns
            * You must put k partitions of the db into the main memory
        * Scan 2: Consolidate global frequent patterns
* **Direct Hashing and Pruning (DHP)**
    * Candidates: a, b, c, d, e
    * Hash entries
        * {ab, ad, ae}
        * {bd, be, de}
        * ...
    * Frequent 1-itemset: a, b, d, e
    * ab is not a candidate 2-itemset if the sum of count of {ab, ad, ae} is below support threshold
* **ECLAT (Exploring Vertical Data Format: ECLAT)**
    * A DFS algo using set intersection
    * Properties of Tid-Lists
        * t(X) = t(Y): X and Y always happend together (e.g. t(ac) = t(d))
        * t(X) in t(Y): transaction having X always has Y (e.g. (t(ac) in t(ce))
        * diffset(ce, e) = t(20)
            * only keeping diffsets can accelerage mining
    * Vertical format:

|item|TidList    |
|----|:---------:|
|  a |10, 20     |
|  b |20, 30     |
|  c |10, 30     |
|  d |10         |
|  e |10, 20, 30 |

* **Frequent Pattern Growth (FPGrowth)**

    * Find frequent single items and partition the db based on each such item
    * Recursively grow frequent patterns by doing the above for each partitioned db (aka conditional db)
    * To facilitate efficient processing, an efficient data structure, **FP-tree**, can be constructed
    * Mining becomes
        * Recursively contruct and mine (conditional) FP-trees
        * Until the resulting FP-tree is empty, or until it contains only one path -- single path will generate all the combinations of its sub-paths, each of which is a frequent pattern
    * p's condition pattern base: ** transformed prefix paths ** of item p
    * use parallel and partition project to scale FPGrowth
    
* **Mining Closed Patterns (CLOSET+)**
    * so many

# Lesson 3: Pattern Evaluation

## What is an interesting pattern?

* Objective:
    * Support, confidence, correlation, ...
* Subjective:
    * Query-based: relevant to a user's particular request
    * Against one's knowledge-base
    * Visualization tools

## List and $\chi^2$

* Measure of dependent/correlated events: **lift**
    * $ lift (B, C) = 
    \dfrac{c(B \implies C)}{s(C)}  = 
    \dfrac{c(B \cup C)}{s(B) \times s(C)}$
        * If list(B, C) = 1, then B and C are independent
        * lift > 1: positively correlated
        * lift < 1: negatively correlated
* Interestingness Measure: $\chi^2$
    * $ \chi^2 = \sum \dfrac{(Observed - Expected)^2}{Expected}$
        * We calculate expected by looking at row sums.
        * $\chi^2$ = 0: independent
        * $\chi^2$ > 0: correlated, either positive or negative so it needs additional test... Just checked observed vs. expected.
* **Null transactions**: transactions that contain neither B nor C

## Null Invariance Measures

* $\chi^2$ and lift are not null-invariant
* AllConf, Jaccard, Cosine, Kulczynski, MaxConf are null-invariant
* Null invariance is important because many transaction sets that contain neither milk nor coffee, which will mess things up when you're trying to compaire milk vs. coffee.

## Comparison of Null-Invariant Measures

* While all five null-invariant measures are comparable when the dataset is very unbalanced, or very balanced.
    * Kulc holds firm and is in balance of both direction imbalances
    * $Kulczynski(A, B) = 
    \frac{1}{2} \big( 
    \dfrac{s(A \cup B)}{s(A)} + \dfrac{s(A \cup B)}{s(B)} \big)$
* Imbalance Ratio with Kulczynski Measure
    * $ IR(A, B) = \dfrac{|s(A) - s(B)|}{s(A) + s(B) - s(A \cup B)}$
* ** The best measure of correlation between dataset is a combination of Kulczynki and imbalance ratio**

# Lesson 4: Mining Diverse Frequent Patterns
## Mining Multi-Level Associations

* Items often form heirarchies:
    * Milk [support = 10%]
        * 2% Milk [support = 6%]
        * Skim Milk [support = 2%]
* How to set min-support thresholds?
    * Uniform min-support across multiple levels? 
    * **Level-reduced min-support**: Items are at the lower level are exepected to have lower support
* Efficient mining: **Shared** multi-level mining
    * Use the lower min-support to pass down the set of candidates
* Multi-level association mining may have many reduncies
    * If lower-level rule can be derived from high-level rule, we remove the low-level rule.
* We can make groups-based 'individualized' min-support.

## Mining Multi-Dimensional Associations

* Single-dimensional rules:
    * buys(X, 'milk') --> buys(X, 'bread')
* Multi-dimensional rules:
    * age(X, '18-25') ^ occupation(X, 'student') --> buys(X, 'coke')
* Hybrid-dimensional rules:
    * age(X, '18-25') ^ buys(X, 'popcorn') --> buys(X, 'coke')
* For categorical data:
    * We can use a data cube
* For quantitative data:
    * We use clustering, etc.
    
## Mining Quantitative Association

* How do we mine numerical attributes?
    * We need to perform **static discretization** based on predefined concept hierarchies
    * We perform clustering to make distance-based association
    * We can also do deviation analysis:
        * e.g. For females, if mean(wage) is different from the all gender mean(wage), that can be interesting.
        * Use statistical tests (Z-test)

## Rare Patterns vs. Negative Patterns

* Rare Patterns
    * Very low support but interesting (e.g. buying Rolex)
    * How to mine? Setting individualized, group-based min-support thresholds for different groups of items
* Negative patterns
    * Negatively correlated: unlikely to happen together (e.g. buying sports car and hybrid car)
    * A support-based definition
        * If itemsets A and B are both frequent but rarely occur together i.e. $sup(A \cup B) << sup (A) \times sup(B) $
            * Then A and B are negatively correlated (lift?)
        * This breaks down again because this is not null-invariant
    * A Kulczynski measure-based definition
        * If itemsets A and B are frequent but (P(A|B) + P(B|A)) / 2 < e where e is a negative pattern threshold
        
## Mining Compressed Patterns

* There are often too many scattered patterns but not so many are meaningful
    * We can just look at closed patterns, but there's too much of a use of support
    * Max patterns meas that we lose too much information
* We can use pattern distance measure
    * $ Dist(P_1, P_2) = 1 - \dfrac{T(P_1) \cap T(P_2)}{T(P_1) \cup T(P_2)} $
    * $ \delta$-clustering: For each pattern P, find all patterns which can be expressed by P and whose distance to P is within $ \delta$ ($ \delta$-cover)

* Redundancy-aware top-k patterns
    * Method: use MMS (Maximal Marginal Significance) for measuring the combined significance of a pattern set.

# Lesson 5: Sequential Pattern Mining

## Sequential Patterns

* This has broad applications:
    * Customer shopping sequences
    * Natural disasters, medical treatments, stocks etc.
    * Click streams
    * Program execution sequences,...
    * Biological sequences: DNA, protein
* Transaction DB, sequence DB vs. time-series DB
* Gapped vs. non-gapped sequential patterns
    * shopping, clicking streams can be gapped
    * biological sequences do care about gaps
* Sequential pattern mining: given a set of sequences, find the **complete set of frequent subsequences** (i.e. satisfying the min_sup threshold)
    * e.g. A **sequence** is < (ef) (ab) (df) c b >
        * An **element** may contain a set of **items** (a.k.a. **events**)
        * Items within an element are unordered and we list them alphabetically
* Algorithm requirement: efficient, scalable, finding complete set, incorporating varios kinds of user-specific constaints
* The Apriori property still holds: if a subsequence s_1 is infrequent, none of s_1's super-sequences can be frequent
* Representative Algorithms:
    * GSP, SPADE, PrefixSpan, CloSpan, Constrain-based sequential pattern mining
    
## GSP: Apriori-Based Sequential Pattern Mining

* Initial candidates: All singleton sequences
* Scan DB once, count support for each candidate
* Generate length-2 candidate sequences
    * Need to add combos (e.g. (ab), (ac), etc.)
    
## SPADE - Sequential Pattern Mining in Vertical Data Format

### IMPORTANT

* Turn transaction DB into vertical series (like my data currently)
* Check which ticks contain "a", "b", etc.
* Check which ticks contain "a" and then which ticks contain "b"
* Combine for bigger patterns

## PrefixSpan - A Pattern-Growth Approach

* **Prefix**: a sub-sequence that looks at the first elements of a sequence. A placeholder is inserted in the **suffix**, or the remaining subsequence, if one element contains other items.
    * e.g. pattern: < a(abc)(ac)d(cf) >, prefix: < aa >, suffix: < (_bc)(ac)d(cf) >

* PrefixSpan Mining:
    * Step 1: Find lenght-1 sequential patterns
    * Step 2: Divid search space and mine each projected DB
* Strengths:
    * No candidate subsequences to be generated
    * Projected DBs keep shrinking
* Costs: Constructing projected DBs
    * Suffixes are lergely repreating in recursive projected DBs
    * When DB can be held in main memory, use pseudo projection
        * No physical copying suffixes
        * Pointer to the sequence (e.g. s|< a >:(, 2) points to < (abc)(ac)d(cf) >
        * Offset of the suffix
    * If it does not fit in memory
        * Physical projection
    * Suggested approach: 
        * Swap to pseudo-projection when the data fits in memory.
        
## CloSpan: Mining Closed Sequential Patterns

* A **closed sequential pattern** s: There exists no superpattern s' s.t. s' contains s, and s' and s have the same support
    * e.g. < abc >: 20, < abcd >: 20 so the latter is closed.
* Why directly mine closed sequential patterns? Same reason we mine close non-sequential patterns.
    * We can use **backward subpattern** and **backward superpattern** pruning to prune redundant search space.